In [ ]:
%pip install -q openai
%pip install -q python-dotenv
%pip install -q requests

In [8]:
from dotenv import load_dotenv
load_dotenv()
import os
API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=API_KEY)

system_message = {
    "role": "system",
    "content": [
        {
            "type": "input_text",
            "text": '''
You are a cutting-edge, futuristic advertising copywriter.
Your task is to generate spoken-style ad scripts that are catchy, vivid, and perfect for TTS-powered video ads.

Given a product or service description, generate a 300–400 words audio-friendly ad script that sounds natural, energetic, and futuristic when spoken aloud.
Use smooth pacing, punchy phrasing, and visual language that pairs well with video.

🔒 Output ONLY the ad script — no instructions, labels, or extra commentary.
No mention of being an AI. No explanations. Just the ad.
            '''
        }
    ]
}

user_message = {
    "role": "user",
    "content": [
        {
            "type": "input_text",
            "text": "Create an ad for the new Abidas Ignite — a lightweight, ultra-responsive running shoe for everyday athletes."
        }
    ]
}

input_format = [system_message, user_message]

# Call the model
response = client.responses.create(
    model="gpt-4.1",
    input=input_format,
    text={"format": {"type": "text"}},
    reasoning={},
    tools=[],
    temperature=1.5,
    max_output_tokens=2048,
    top_p=1,
    store=True
)

print(response.output_text)

In [ ]:
import openai

input = response.output_text

response = openai.audio.speech.create(
    model="tts-1",
    voice="nova",
    input=input
)

# Save to file (optional)
with open("abidas_ad.mp3", "wb") as f:
    f.write(response.read())

print("Audio saved to output.mp3")


In [ ]:
import os
import requests
from openai import OpenAI

client = OpenAI(api_key=API_KEY)

# Ensure output directory exists
os.makedirs("ad_images", exist_ok=True)

prompts = [
    "A futuristic city skyline glowing with neon lights and digital billboards, sleek drones flying above, cinematic lighting",
    "A high-tech smart device floating in a minimal white space, glowing UI elements and holographic effects, product-focused shot",
    "A young, diverse person wearing augmented reality glasses, standing in a bright, digital world with floating data and holograms",
    "Abstract representation of innovation — glowing lines, circuits, and energy pulses forming the shape of a brain or lightning bolt",
    "Close-up of a human hand interacting with a holographic touchscreen interface, surrounded by futuristic data visualizations",
    "Futuristic fashion runway with models wearing tech-integrated clothing, glowing fabrics, and AI-driven accessories",
    "A dynamic motion-blur shot of a product launch with dramatic lighting, spotlight on the product with floating particles",
    "A surreal digital landscape showing transformation — old tech dissolving into new glowing devices, metaphor for innovation",
    "Energetic abstract background with 3D shapes, bright gradients, and lens flares to use as ad background or transition scene",
    "A peaceful futuristic home setting with smart devices seamlessly working in harmony — clean, warm lighting, family-friendly vibe"
]

for i, prompt in enumerate(prompts):
    result = client.images.generate(
        model="dall-e-2",
        prompt=prompt,
        size="1024x1024",
        n=1,
    )

    image_url = result.data[0].url
    image_data = requests.get(image_url).content

    with open(f"ad_images/image_{i+1}.png", "wb") as f:
        f.write(image_data)

    print(f"Saved image {i+1}: {image_url}")
